In [ ]:
import os
from flask import Flask, render_template, request, redirect, url_for, session, flash
import mysql.connector
from mysql.connector import Error
from werkzeug.security import generate_password_hash, check_password_hash
from flask_cors import CORS, cross_origin
from pyngrok import ngrok, conf as ngrok_conf

NGROK_API_KEY="35grkTvhdxBmJ2YHaqbdYWQaDQi_2hMvmPt4jAEJJxKaQJ9tS"
ngrok_conf.get_default().auth_token = NGROK_API_KEY

# Inisialisasi flask
app = Flask(__name__)

# Pengaturan secret key untuk session
app.secret_key = os.urandom(24) # Secret key untuk menjaga keamanan sesi pengguna

# Koneksi CORS
cors = CORS(app) # Mengaktifkan CORD untuk memperbolehkan akses dari sumber berbeda

# Fungsi untuk membuat koneksi ke database
def create_connection():
    try:
        # Membuat koneksi ke database MySQL
        conn = mysql.connector.connect(
            host="localhost",      # Host database
            user="root",           # User untuk Login ke MySQL
            passwd="",             # Password MySQL
            database="belajar_web" # Nama database yang ingin digunakan
        )
        if conn.is_connected():
            return conn # Mengembalikan objek
    except Error as e:
        print(f"Error while connecting to MySQL: {e}")
    return None

# ROUTE LOGIN DAN REGISTRASI
@app.route('/')
@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        # Mengambil data dari form login
        username = request.form['username']
        password = request.form['password']
        
        # Buat koneksi ke database
        conn = create_connection()
        if conn:
            # Membuat cursor untuk melakukan query
            cursor = conn.cursor(dictionary=True)
            cursor.execute("SELECT * FROM users WHERE username = %s and password = %s", (username, password,))
            user = cursor.fetchone() # Mengambil satu hasil dari query
            
            cursor.close() # Menutup cursor
            conn.close()   # Menutup koneksi database
            
            # Mengecek apakah user ditemukan dan password sesuai
            if user:
                session['loggedin'] = True 
                session['username'] = user ['username']
                session['id'] = user['id'] 
                return redirect(url_for('dashboard')) 
            else:
                flash('Invalid username or password!', 'danger') # Menampilkan pesan error jika gagal
        else:
            flash('Could not connect to the database. Please try again later.', 'danger') 
            
    return render_template('login.html') # Render halaman login

# Route untuk halaman registrasi
@app.route('/register', methods=['GET', 'POST'])
def register():
    if request.method == 'POST':
        # Mengambil data dari form registrasi
        username = request.form['username']
        email = request.form['email']
        password = request.form['password']
        confirm_password = request.form['confirm-password']
        
        # Mengecek apakah password dan konfirmasi password sama
        if password != confirm_password:
            flash('Passwords do not match!', 'danger') # Menampilkan pesan error jika password tidak cocok
            return redirect(url_for('register'))
        
        # Buat koneksi ke database
        conn = create_connection()
        if conn:
            cursor = conn.cursor() # Mmebuat cursor
            try:
                # Melakukan query untuk menyimpan data user ke database
                cursor.execute("INSERT INTO users (username, email, password) VALUES (%s, %s, %s)", 
                               (username, email, password,))
                conn.commit() # Commit perubahan ke database
                flash('Registration successful. Please log in.', 'success') # Pesan jika registrasi berhasil
            except mysql.connector.IntegrityError:
                flash('Username already exists!', 'danger') # Pesan error jika username sudah ada
            finally:
                cursor.close() # Menutup Cursor
                conn.close()   # Menutup koneksi database
            return redirect(url_for('login')) # Redirect ke halaman login
        else:
            flash('Could not connect to the database. Please try again later.', 'danger') 
            
    return render_template('register.html')

# -----------------------------------------------------------
# ROUTE DASHBOARD DAN PROFILE
# -----------------------------------------------------------
@app.route('/dashboard')
def dashboard():
    if 'loggedin' in session:
        # Jika user sudah login, render halaman dashboard
        username = session['username']
        return render_template('dashboard.html', username=username)
    else:
        # Jika user belum login, redirect ke halaman login
        flash('Please login to view this!', 'danger')
        return redirect(url_for('login'))
    
@app.route('/profile')
def profile():
    #Pengecekan session id
    if 'id' not in session:
        flash('Please logged in first.', 'danger')
        return redirect(url_for('login'))
    
    #Pengecekan session id
    conn = create_connection()
    if conn:
        cursor = conn.cursor(dictionary=True) #membuat cursor
        try:
            #Mengambil data pengguna berdasarkan sesi pengguna yang login
            cursor.execute('SELECT * FROM users WHERE id = %s', (session['id'],))
            user = cursor.fetchone()
            #Render halaman profil dengan data pengguna
            return render_template('profile.html', 
                                   username=user['username'], 
                                   email=user['email'],
                                   level=user['level'])
        
        finally:
            cursor.close() #Menutup cursor
            conn.close() #Menutup koneksi database
    return redirect(url_for('login')) #Jika gagal terhubung, redirect ke login

@app.route('/update_password', methods=['POST'])
def update_password():
    current_password = request.form['current_password'] 
    new_password = request.form['new_password'] 
    confirm_password = request.form['confirm_password'] 

    #Menghubungkan ke database
    conn = create_connection() 
    if conn: 
        cursor = conn.cursor(dictionary=True) #Membuat cursor 
        try: 
            #Mengambil data dari database
            cursor.execute('SELECT * FROM users WHERE id = %s', (session['id'],)) 
            user = cursor.fetchone() 
     
            #Verifikasi password lama
            if (user['password'] != current_password): 
                flash('Current password is incorrect.', 'danger') 
                return redirect(url_for('profile')) 

            #Verifikasi konfimasi password baru
            if new_password != confirm_password: 
                flash('New passwords do not match.', 'danger') 
                return redirect(url_for('profile')) 

            #Update password dengan password baru yang di-hash
            cursor.execute('UPDATE users SET password = %s WHERE id = %s', (new_password, session['id'])) 
            conn.commit() 
            
            flash('Password has been updated successfully.', 'success') 
            return redirect(url_for('profile')) 
        except mysql.connector.IntegrityError: 
            flash('Error updating password. Please try again.', 'danger')#Menangani error
            return redirect(url_for('profile')) 
        finally: 
            cursor.close()#Menutup cursor
            conn.close() #Menutup koneksi database
    else: 
           flash('Database connection failed.', 'danger')#Pesan error jika gagal terhubung
    return redirect(url_for('profile')) 

#Tampilkan tabel mahasiswa dengan pagination dan pencarian
@app.route('/mahasiswa') 
def mahasiswa(): 
    
     #Menghubungkan ke database       
    conn = create_connection() 
    cursor = conn.cursor(dictionary=True) 
    
    #Pagination
    per_page = 5  #Jumlah data per halaman
    page = request.args.get('page', 1, type=int) 
    offset = (page - 1) * per_page 
        
    # Pencarian
    search = request.args.get('search','')
        
    if search: 
        query = "SELECT * FROM mahasiswa WHERE nama LIKE %s LIMIT %s OFFSET %s"
        cursor.execute(query, (f"%{search}%", per_page, offset))
                    
    else: 
        query = "SELECT * FROM mahasiswa LIMIT %s OFFSET %s" 
        cursor.execute(query, (per_page, offset))
           
    data = cursor.fetchall()

    #Menghitung total data untuk pagination
    if search:
        cursor.execute("SELECT COUNT(*) FROM mahasiswa WHERE nim LIKE %s OR nama LIKE %s OR jurusan LIKE %s",
                        (f"%{search}%", f"%{search}%", f"%{search}%"))
    else:
        cursor.execute("SELECT COUNT(*)FROM mahasiswa")
        
    total = cursor.fetchone()['COUNT(*)']
    cursor.close()
    conn.close()
        
    return render_template('mahasiswa.html', data=data, page=page, per_page=per_page, total=total, search=search)

#Menambah mahasiswa baru
@app.route('/mahasiswa/tambah', methods=['POST', 'GET'])
def tambah_mahasiswa():
     if request.method == 'POST':
        nim = request.form['nim']
        nama = request.form['nama']
        jurusan = request.form['jurusan']
        angkatan = request.form['angkatan']

        conn = create_connection()
        cursor = conn.cursor()
        cursor.execute('INSERT INTO mahasiswa(nim,nama,jurusan,angkatan) VALUES (%s,%s,%s,%s)',
                     (nim, nama, jurusan, angkatan))
        conn.commit()
        cursor.close()
        conn.close()
        
        flash('Data mahasiswa berhasil ditambahkan!','success')
        return redirect(url_for('mahasiswa'))
     return render_template('tambah_mahasiswa.html')


@app.route('/mahasiswa/edit/<int:id>', methods=['POST', 'GET'])
def edit_mahasiswa(id):
    #Menghubungkan ke database
    conn = create_connection()
    cursor = conn.cursor(dictionary=True)
    cursor.execute('SELECT * FROM mahasiswa WHERE id = %s',(id,))
    mahasiswa = cursor.fetchone()
    
    if request.method == 'POST':
        nim = request.form['nim']
        nama = request.form['nama']
        jurusan = request.form['jurusan']
        angkatan = request.form['angkatan']

        cursor.execute('UPDATE mahasiswa SET nim = %s, nama = %s, jurusan = %s, angkatan = %s WHERE id = %s',
                      (nim, nama, jurusan, angkatan, id))
        conn.commit()
        cursor.close()
        conn.close()
        
        flash('Data mahasiswa berhasil diupdate!', 'success')
        return redirect(url_for('mahasiswa'))
  
    cursor.close()
    conn.close()
    return render_template('edit_mahasiswa.html', mahasiswa=mahasiswa)

# Menghapus data mahasiswa
@app.route('/mahasiswa/hapus/<int:id>', methods=['POST'])
def hapus_mahasiswa(id):
    # Menghubungkan ke database
    conn = create_connection()
    cursor = conn.cursor()
    cursor.execute('DELETE FROM mahasiswa WHERE id = %s', (id,))
    conn.commit()
    cursor.close()
    conn.close()
    
    flash('Data mahasiswa berhasil dihapus', 'success')
    return redirect(url_for('mahasiswa'))

# Route untuk logout
@app.route('/logout')
def logout():
    # Menghapus session user yang sedang login
    session.pop('loggedin', None)
    session.pop('username', None)
    flash('You have been logged out successfully.', 'success') # Menampilkan pesan sukses
    return redirect(url_for('login')) # Redirect ke halaman login

# Fungsi utama untuk menjalankan aplikasi flask
if __name__== '__main__':
    port = int(os.environ.get("PORT", 5000))
    public_url = ngrok.connect(port, proto ="http").public_url
    print(f"Public URL (ngrok):{public_url}")
    
    app.run() # Menjalankan server flask dalam mode debug (untuk pengembangan)

Public URL (ngrok):https://marbly-interproportional-guillermo.ngrok-free.dev
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [19/Nov/2025 16:20:56] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Nov/2025 16:21:05] "POST /login HTTP/1.1" 302 -
127.0.0.1 - - [19/Nov/2025 16:21:05] "GET /dashboard HTTP/1.1" 200 -
127.0.0.1 - - [19/Nov/2025 16:21:11] "GET /profile HTTP/1.1" 200 -
127.0.0.1 - - [19/Nov/2025 16:21:55] "GET /logout HTTP/1.1" 302 -
127.0.0.1 - - [19/Nov/2025 16:21:55] "GET /login HTTP/1.1" 200 -
127.0.0.1 - - [19/Nov/2025 16:21:57] "GET /register HTTP/1.1" 200 -
127.0.0.1 - - [19/Nov/2025 16:22:27] "POST /register HTTP/1.1" 302 -
127.0.0.1 - - [19/Nov/2025 16:22:27] "GET /login HTTP/1.1" 200 -
127.0.0.1 - - [19/Nov/2025 16:22:44] "POST /login HTTP/1.1" 302 -
127.0.0.1 - - [19/Nov/2025 16:22:44] "GET /dashboard HTTP/1.1" 200 -
127.0.0.1 - - [19/Nov/2025 16:22:48] "GET /profile HTTP/1.1" 200 -
127.0.0.1 - - [19/Nov/2025 16:23:00] "GET /mahasiswa HTTP/1.1" 200 -
127.0.0.1 - - [19/Nov/2025 16:23:04] "GET /mahasiswa/tambah HTTP/1.1" 200 -
1